# DAT565 Introduction to Data Science and AI 
## 2023-2024, LP2
## Assignment 4: Spam classification using Naïve Bayes 
This assignment has three obligatory questions. Questions 4-5 are optional and will not be graded.

The exercise takes place in this notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical. 

*Tips:* 
* You can execute certain Linux shell commands by prefixing the command with a `!`. 
* You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results, the second you can use to write code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets. 

You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location: 
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages. 
-	hard-ham: non-spam messages more difficult to differentiate 
-	spam: spam messages 

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.** 

If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer. Note that if you are using Windows, you can instead use [7zip](https://www.7-zip.org/download.html) to decompress the data (you will have to modify the cell below).

**What to submit:** 
* Convert the notebook to a PDF file by compiling it, and submit the PDF file. 
* Make sure all cells are executed so all your code and its results are included. 
* Double-check that the PDF displays correctly before you submit it.

In [ ]:
# download and extract the data
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2
!tar -xjf 20021010_easy_ham.tar.bz2
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2

The data is now in the following three folders: `easy_ham`, `hard_ham`, and `spam`. You can confirm this via the following command:

In [ ]:
!ls -lah

### 1. Preprocessing: 
Note that the email files contain a lot of extra information, besides the actual message. Ignore that for now and run on the entire text (in the optional part further down, you can experiment with filtering out the headers and footers). 
1.	We don’t want to train and test on the same data (it might help to reflect on **why** ,if you don't recall). Split the spam and ham datasets into a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`). Use `easy_ham` for quesions 1 and 2.


In [1]:
# write your pre-processing code here
import sklearn as sk
from sklearn.model_selection import train_test_split
import os
import collections
hamFileList= os.listdir("./easy_ham")
# collections.shuffle(hamFileList)
spamFileList= os.listdir("./spam")
hamtrain,hamtest = train_test_split(hamFileList,test_size=0.3)
print(len(hamtrain))

print(len(hamtest))

spamtrain, spamtest = train_test_split(spamFileList,test_size=0.3)
print(len(spamtrain))

print(len(spamtest))
print(spamtest)

1785
766
350
151
['0439.2c6d4d3e8b817c5315a1c61e4eeb7ee9', '0372.216f90ef52558ed24402e192586a40e8', '0280.2507969221ea95a019506366f6c361d8', '0011.2a1247254a535bac29c476b86c708901', '0099.c4ff6dba0a5177d3c7d8ef54c8920496', '0131.0b7281078874ca88f95d6fdf5d905d50', '0423.3b9b432fa25a7ac8c494db8d2e42cc73', '0199.955edee89f34960c033c4d1072841356', '0091.113ec7122d4046a2754bcf70b9fb5299', '0338.033c0109da096486c7d797cccd2c3198', '0326.80f15e07265a22b78068bab5b56b01c7', '0166.a2e4d6ec3078b619ca38927ca69fc94d', '0203.beb1b157fc74672074061434cc7bad3c', '0197.6968d98720065059247cefe4e5bcd192', '0301.ad155a30cca1f9d16e75e8934030edae', '0051.374f4d4300a5d39544b2f052e7a9429d', '0204.33e3cd4e0ad791304e554bb259bda53e', '0319.e4a20802d12937998f3b3bf805362a3f', '0415.e241b6184464107168656739bf96c6b9', '0335.9822e1787fca0741a8501bdef7e8bc79', '0080.77af9ca7f967f055062aade45001129e', '0270.d50e186af7a00114ad967b8f77b70338', '0244.7e5d917c8a76d52cc694c5cf8ab8497d', '0047.376bd7728ee94b32bc23429d9c51bae5'

### 2. Write a Python program that: 
1.	Uses the four datasets from Question 1 (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`).
2.	Trains a Naïve Bayes classifier (use the [scikit-learn library](https://scikit-learn.org/stable/)) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and False Negative rates on the `hamtest` and `spamtest` datasets. You can use `CountVectorizer` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)) to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifiers available in *scikit-learn* ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Here, you will test two of these classifiers that are well suited for this problem:
- Multinomial Naive Bayes
- Bernoulli Naive Bayes.

Please inspect the documentation to ensure input to the classifiers is appropriate before you start coding. You may have to modify your input.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

def extractFeatureFromFile(filePathList:[],isSpam:bool):
    result= pd.DataFrame()
    vectorizer = CountVectorizer(lowercase=True , decode_error="ignore")
    for filePath in filePathList:
        if isSpam:
            file= open("./spam/"+filePath, "r")
        else:
            file = open("./easy_ham/",+filePath, "r")
            
        X = vectorizer.fit_transform(file)
        new_row = pd.DataFrame({"feature":[X],"isSpam":1 if isSpam else 0})
        result = result.append(new_row)
    return result

    


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

def extractFeatureFromFile1(filePathList:[],isSpam:bool):
    result= pd.DataFrame()
    for filePath in filePathList:
        if isSpam:
            file= open("./spam/"+filePath, "rb")
        else:
            file = open("./easy_ham/",+filePath, "rb")
            
        new_row = pd.DataFrame({"feature":file,"isSpam":1 if isSpam else 0})
        result = result.append(new_row)
    return result

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
def extractFeatureFromFile2(filePathList, isSpam):
    result = pd.DataFrame()
    all_text = []

    for filePath in filePathList:
        if isSpam:
            file_path = "./spam/" + filePath
        else:
            file_path = "./easy_ham/" + filePath

        # with open(file_path, "r",encoding='latin-1') as file:
        #     data = file.read()
        #     data = data.replace('\n', ' ')
        #     all_text.append(data)
        with open(file_path, "r",encoding='iso-8859-1') as file:
            data = file.read()
            data = data.replace('\n', ' ')
            all_text.append(data)
    vectorizer = CountVectorizer(ngram_range=(1, 2)).fit(all_text)

    for text in all_text:
        X = vectorizer.transform([text])
        new_row = pd.DataFrame({"feature": [X], "isSpam": 1 if isSpam else 0})
        result = result.append(new_row, ignore_index=True)

    return result

In [10]:
# write your code here
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import  confusion_matrix
spamtrain_df = extractFeatureFromFile2(spamtrain,True) #[features,1]
hamtrain_df = extractFeatureFromFile2(hamtrain,False)  #[features,0]
spamtest_df = extractFeatureFromFile2(spamtest,False)  #[features,0]
hamtest_df = extractFeatureFromFile2(hamtest,False)  #[features,0]
train_df= pd.concat([spamtrain_df,hamtrain_df], ignore_index=True)
test_df= pd.concat([spamtest_df,hamtest_df], ignore_index=True)





multinomial_classifier = MultinomialNB()
multinomial_classifier.fit(train_df["feature"], train_df["isSpam"])

bernoulli_classifier = BernoulliNB()
bernoulli_classifier.fit(train_df["feature"], train_df["isSpam"])

y_pred_multinomial = multinomial_classifier.predict(test_df["feature"])
y_pred_bernoulli = bernoulli_classifier.predict(test_df["feature"])


bernoulli_classifier_confusion = confusion_matrix(test_df["isSpam"], y_pred_bernoulli, labels=['ham', 'spam'])
bernoulli_classifier_true_positive_rate = bernoulli_classifier_confusion[0, 0] / (bernoulli_classifier_confusion[0, 0] + bernoulli_classifier_confusion[0, 1])
bernoulli_classifier_false_negative_rate = bernoulli_classifier_confusion[1, 0] / (bernoulli_classifier_confusion[1, 0] + bernoulli_classifier_confusion[1, 1])

multinomial_classifier_confusion = confusion_matrix(test_df["isSpam"], y_pred_multinomial, labels=['ham', 'spam'])
multinomial_classifier_true_positive_rate = multinomial_classifier_confusion[0, 0] / (multinomial_classifier_confusion[0, 0] + multinomial_classifier_confusion[0, 1])
multinomial_classifier_false_negative_rate = multinomial_classifier_confusion[1, 0] / (multinomial_classifier_confusion[1, 0] + multinomial_classifier_confusion[1, 1])


/var/folders/fm/z_773gfs2d3cd8ch6mq3_s3r0000gn/T/ipykernel_984/572207603.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(new_row, ignore_index=True)


FileNotFoundError: [Errno 2] No such file or directory: './easy_ham/0439.2c6d4d3e8b817c5315a1c61e4eeb7ee9'

### 3. Run on hard ham:
Run the two models from Question 2 on `spam` versus `hard-ham`, and compare to the `easy-ham` results.

In [ ]:
# code to report results here

### 4.	OPTIONAL - NOT MARKED: 
To avoid classification based on common and uninformative words, it is common practice to filter these out. 

**a.** Think about why this may be useful. Show a few examples of too common and too uncommon words. 

**b.** Use the parameters in *scikit-learn*’s `CountVectorizer` to filter out these words. Update the program from Question 2 and run it on `easy-ham` vs `spam` and `hard-ham` vs `spam`. Report your results.

In [ ]:
# write your code here

### 5. OPTIONAL - NOT MARKED: Further improving performance
Filter out the headers and footers of the emails before you run on them. The format may vary somewhat between emails, which can make this a bit tricky, so perfect filtering is not required. Run your program again and answer the following questions: 
- Does the result improve from those obtained in Questions 3 and 4? 
- What do you expect would happen if your training set consisted mostly of spam messages, while your test set consisted mostly of ham messages, or vice versa? 
- Look at the `fit_prior` parameter. What does this parameter mean? Discuss in what settings it can be helpful (you can also test your hypothesis). 

In [ ]:
# write your code here